In [1]:
import sys
sys.path.append('..')
from utils.pytorch_helper import *
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, fasterrcnn_mobilenet_v3_large_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.amp import GradScaler
from torch.utils.data import DataLoader
# with help of this tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
model_type = "mobilenet" # "resnet50"

In [2]:
# Train Dataset
images_dir = "../../data/merged/images/train"
annotations_dir = "../../data/merged/labels/train"
train_dataset = LicensePlateDataset(images_dir, annotations_dir)
#train_subset = get_subset(train_dataset, fraction=0.03) # for testing
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, collate_fn=collate_fn)

# Validation Dataset
validation_images_dir = "../../data/merged/images/val"
validation_annotation_dir = "../../data/merged/labels/val"
val_dataset = LicensePlateDataset(validation_images_dir, validation_annotation_dir)
#val_subset = get_subset(val_dataset, fraction=0.2) # for testing
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [3]:
# use pretrained weights
model = fasterrcnn_resnet50_fpn_v2(weights="DEFAULT") if model_type == "resnet50" else fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Load Model with backbone {model_type} to {device}")

# scaler helps me with fitting larger batches in the GPU memory, lowers the precision of floats
scaler = GradScaler("cuda") 
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.002, momentum=0.9, weight_decay=0.0001)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

Load Model with backbone mobilenet to cuda


In [ ]:
best_mAP50 = 0.0
best_mAP50_95 = 0.0
stopping_counter = 0
patience = 1
num_epochs = 10
start_epoch = 0

continue_training = True
if continue_training:
    # Faster RCNN Resnet50:
    # Epoch 0 Validation: mAP50 = 0.9146, mAP50-95 = 0.6440
    # Epoch 1 Validation: mAP50 = 0.9233, mAP50-95 = 0.6387
    # -> Epoch 2 Validation: mAP50 = 0.9268, mAP50-95 = 0.6613
    # Faster RCNN Mobilenet V3:
    # Epoch 0 Validation: mAP50 = 0.8868, mAP50-95 = 0.6464
    # Epoch 1 Validation: mAP50 = 0.8908, mAP50-95 = 0.6532
    # -> Epoch 2 Validation: mAP50 = 0.8997, mAP50-95 = 0.6668
    # Epoch 3 Validation: mAP50 = 0.8878, mAP50-95 = 0.6806
    checkpoint = torch.load(f"../../models/best_fasterrcnn_{model_type}.pth", weights_only=False)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    lr_scheduler.load_state_dict(checkpoint["lr_scheduler_state_dict"])
    start_epoch = checkpoint.get("epoch", -1) + 1
    best_mAP50 = checkpoint.get("mAP50", 0)
    best_mAP50_95 = checkpoint.get("mAP50-95", 0)
    print(f"Checkpoint loaded! Start epoch = {start_epoch}")

Checkpoint loaded! Start epoch = 3


In [ ]:
for epoch in range(start_epoch, num_epochs):
    train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch, scaler)
    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}")
    
    lr_scheduler.step()
    
    metrics = evaluate(model, val_loader, device, fasterrcnn_predict) 
    mAP50 = metrics['mAP50']
    mAP50_95 = metrics['mAP50-95']
    print(f"Epoch {epoch} Validation: mAP50 = {mAP50:.4f}, mAP50-95 = {mAP50_95:.4f}")
    if mAP50 > best_mAP50 or mAP50_95 > best_mAP50_95:
        best_mAP50 = max(mAP50, best_mAP50)
        best_mAP50_95 = max(mAP50_95, best_mAP50_95)
        stopping_counter = 0
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "lr_scheduler_state_dict": lr_scheduler.state_dict(),
            "mAP50": best_mAP50,
            "mAP50-95": best_mAP50_95
        }, f"../../models/best_fasterrcnn_{model_type}.pth")
        print(f"Model saved at epoch {epoch}.")
    else:
        stopping_counter += 1
    
    if stopping_counter == patience:
        print("Early stopping triggered. Stop training.")
        break